# AA_crossmatch_SDSS_to_ALL

Taking the S82 QSO from SDSS, CRTS, PS1, PTF : <a id='toc'></a>
    
   * cross-match, regardless of filter:
    * [SDSS-PS1](#sdss_ps1)
    * [SDSS-PTF](#sdss_ptf)
    * [SDSS-CRTS](#sdss_crts)
    * [SDSS-CRTS-PTF-PS1](#combine)
   * per object : time, mag, magerr, filter, survey
   * simulate 'perfect' case: very long, very dense light curves
   * sample at cadence of 1) SDSS, 2) SDSS+CRTS,  3) SDSS+CRTS+PTF , 
       4)SDSS+CRTS+PTF+PS1 , fitting with Celerite MAP Jeff1,2 
   * show that as we add more data, the retrieved tau gets better 
    

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import pandas as pd 
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import  hstack
from astropy.table import  vstack
from astropy.table import Column
from astropy.table import join

## SDSS + PS1 <a id='sdss_ps1'></a>

In [2]:
ps1_qso_schneider = os.listdir('../raw_data/schneider/')

# grab ra, dec from filenames 
ra_ps1 = np.array([coord[:9] for coord in ps1_qso_schneider])
dec_ps1 = np.array([coord[9:] for coord in ps1_qso_schneider])

# make into AstroPy - digestible format.. 
ra_ps1_split = [ra[:2] + ' '+ ra[2:4] + ' '+ra[4:] for ra in ra_ps1]
dec_ps1_split = [dec[:3]+ ' '+ dec[3:5]+ ' ' + dec[5:] for dec in dec_ps1]

# combining the two.. 
radec_ps1_split = np.zeros_like(ra_ps1_split, dtype='U32')
for i in range(len(ra_ps1_split)):
    radec_ps1_split[i] = ra_ps1_split[i] + ' ' + dec_ps1_split[i]

c = SkyCoord(radec_ps1_split, frame='icrs', unit=(u.hourangle, u.deg)) 

# Check how many SDSS S82 DR9 QSOs have PS1 matches 
File = '../../CRTS_PROJECT/catalogs_SDSS/DB_QSO_S82.dat'
colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']

# 9258 QSO ....
SDSS_DB_QSO= Table.read(File, format='ascii', names = colnames)
sdss_catalog = SkyCoord(ra=SDSS_DB_QSO['ra']*u.degree, dec=SDSS_DB_QSO['dec']*u.degree)


# Match Schneider PS1 to SDSS S82 DR7 : 
# better : match SDSS to PS1,  
# because there is less 
idx, sep2d, dist3d = sdss_catalog.match_to_catalog_sky(c) 
SDSS_DB_QSO['d2d_arcsec']=sep2d.value*3600

ps1id = np.asarray(ps1_qso_schneider)[idx]
ps1ra = c[idx].ra.deg
ps1dec = c[idx].dec.deg
ps1 = Table([ps1id, ps1ra, ps1dec], names=['ps1id', 'ps1ra', 'ps1dec'])

sdss_ps1 = hstack([SDSS_DB_QSO, ps1])



In [3]:
SDSS_DB_QSO

dbID,ra,dec,SDR5ID,M_i,M_i_corr,redshift,mass_BH,Lbol,u,g,r,i,z,Au,d2d_arcsec
int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
70,2.169302,1.238649,301,-23.901,-24.181,1.073,0.0,0.0,20.793,20.469,20.197,20.04,20.0,0.116,0.0520399207824
98,1.091028,0.962126,144,-23.399,-23.576,0.7867,0.0,0.0,20.79,20.183,19.849,19.818,19.43,0.183,0.114152245771
233,0.331289,0.17723,58,-24.735,-25.058,1.6199,0.0,0.0,20.892,20.554,20.431,20.199,20.099,0.154,0.143164141458
1018,1.364696,-0.098956,190,-23.121,-24.045,0.6125,0.0,45.433,20.098,19.722,19.784,19.485,19.541,0.178,0.113498399727
1310,0.221552,-0.292485,36,-26.451,-26.974,2.7563,9.361,46.76,20.707,19.663,19.61,19.705,19.529,0.174,0.0591591407989
1327,0.28137,0.686984,49,-24.917,-25.288,1.3988,8.817,46.071,20.213,20.105,19.815,19.666,19.749,0.139,0.138633528526
1540,2.607954,0.913245,370,-27.263,-27.856,2.8473,0.0,0.0,21.346,19.15,19.093,18.944,18.758,0.122,0.157425447116
1580,1.669492,0.643952,229,-24.427,-24.898,1.3383,8.666,45.862,20.732,20.572,20.13,20.079,20.231,0.207,0.124207184069
2113,4.850273,-0.276395,699,-25.251,-25.695,1.769,0.0,0.0,19.956,20.066,20.152,19.886,19.618,0.147,0.0856719350943


In [5]:
matchRadiusArcsec = 0.5
fname = 'SDSS_DB9_PS1_QSO_database.txt'
mask = sdss_ps1['d2d_arcsec'] < matchRadiusArcsec
print('Of ', len(SDSS_DB_QSO) , ' SDSS DB9 QSO, ', 
      np.sum(mask), 
    ' has a PS1 match within ', matchRadiusArcsec, ' arcseconds ')

print('We save these to ', fname)
sdss_ps1[mask].write(fname, format='ascii', overwrite=True)

Of  9258  SDSS DB9 QSO,  9248  has a PS1 match within  0.5  arcseconds 
We save these to  SDSS_DB9_PS1_QSO_database.txt


## SDSS-PTF <a id='sdss_ptf'></a>
[Go to top](#toc)

NOTE : Those that were already cross-matched earlier for the CRTS_PROJECT involved only PTF Quasars that hade more than 10 obs a day. That significantly reduced the number ( from 6471 quasars with 159641 epochs in ../IPAC_output_tables/Ipac_PTF_lightcurves_QSO_S82.tbl , to only 2753 quasars stored in ../../CRTS_PROJECT/data_products/PTF_SDSS_cross_matched_qso_DB9_catalog.dat ...)

Thus I use the original ptf_lc file, and do the cross-match by SDSS ra , called 'ra' in PTF IPAC light curves file. 

In [4]:
ptf_lcs =  Table.read('../../CRTS_PROJECT/IPAC_output_tables/\
Ipac_PTF_lightcurves_QSO_S82.tbl', 
                     format='ipac')


In [166]:
ptf_lcs

cntr_01,dist_x,pang_x,ra_01,dec_01,obsmjd,mag_autocorr,magerr_auto,oid,ra,dec,fid,transient_flag,astrometricrms,nobs,ngoodobs,nbestobs,referencemag,referencemagerr,medianmag,bestmedianmag,meanmag,bestmeanmag,magrms,bestmagrms,sid,s_ra,s_dec,ptffield,ccdid,hmjd,nid,mag_auto,limitmag,absphotzp,fwhmsex,fwhm_image,qaflags,flags,photcalflag,goodflag
,arcsec,deg,degrees,degrees,days,mag,mag,,degrees,degrees,,,degrees,,,,mag,mag,mag,mag,mag,mag,mag,mag,,degrees,degrees,,,days,,mag,mag,mag,arcsec,pixels,,,"0=No, 1=Yes","0=No, 1=Yes"
int64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,int64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,int64,float64,int64,float64,float64,float64,float64,float64,int64,int64,int64,int64
1,0.0,0.0,6.968328,0.437734,56903.3404372,18.747,0.04,26991060002225,6.968328,0.437734,1,0,4.777896e-07,125,125,123,18.717,0.006,18.746,18.748,18.743,18.743,0.06,0.06,21305664001544,6.968329,0.437729,2699,6,56903.3455472,1968,18.76,21.93,27.896,2.45,2.713,0,131072,0,1
1,0.0,0.0,6.968328,0.437734,56903.3701872,18.739,0.04,26991060002225,6.968328,0.437734,1,0,4.777896e-07,125,125,123,18.717,0.006,18.746,18.748,18.743,18.743,0.06,0.06,21305689001545,6.968338,0.43773,2699,6,56903.3752972,1968,18.749,21.982,27.901,2.35,2.558,0,131072,0,1
1,0.0,0.0,6.968328,0.437734,56904.3353572,18.766,0.047,26991060002225,6.968328,0.437734,1,0,4.777896e-07,125,125,123,18.717,0.006,18.746,18.748,18.743,18.743,0.06,0.06,21299870001227,6.968337,0.43772,2699,6,56904.3405072,1969,18.784,21.467,27.896,2.45,2.968,0,131072,0,1
1,0.0,0.0,6.968328,0.437734,56904.3635872,18.721,0.043,26991060002225,6.968328,0.437734,1,0,4.777896e-07,125,125,123,18.717,0.006,18.746,18.748,18.743,18.743,0.06,0.06,21299891001474,6.96832,0.437738,2699,6,56904.3687372,1969,18.728,21.97,27.901,2.45,2.778,0,131072,0,1
1,0.0,0.0,6.968328,0.437734,56951.2452872,18.741,0.052,26991060002225,6.968328,0.437734,1,0,4.777896e-07,125,125,123,18.717,0.006,18.746,18.748,18.743,18.743,0.06,0.06,21903907001390,6.968324,0.43772,2699,6,56951.2507372,2013,18.736,21.813,27.82,2.35,2.646,0,131072,0,1
1,0.0,0.0,6.968328,0.437734,56951.2733472,18.785,0.053,26991060002225,6.968328,0.437734,1,0,4.777896e-07,125,125,123,18.717,0.006,18.746,18.748,18.743,18.743,0.06,0.06,21903932001362,6.968328,0.437736,2699,6,56951.2787872,2013,18.777,21.789,27.821,2.45,2.796,0,131072,0,1
1,0.0,0.0,6.968328,0.437734,56946.2431172,18.73,0.044,26991060002225,6.968328,0.437734,1,0,4.777896e-07,125,125,123,18.717,0.006,18.746,18.748,18.743,18.743,0.06,0.06,21824771001223,6.968352,0.437753,2699,6,56946.2487072,2008,18.746,21.592,27.845,2.55,3.13,0,131072,0,1
1,0.0,0.0,6.968328,0.437734,56956.2378572,18.662,0.276,26991060002225,6.968328,0.437734,1,0,4.777896e-07,125,125,123,18.717,0.006,18.746,18.748,18.743,18.743,0.06,0.06,21999993001204,6.968332,0.437743,2699,6,56956.2431172,2018,18.083,21.036,27.25,2.55,2.86,0,131072,0,1


I use the 'ra' key that corresponds to  the SDSS ra, since ra_01 : the PTF ra , may differ from chip to chip, just as oid . For these quasars there seems to be only one edge-case ( number of unique ra is 6471, and oid : 6472, ra_01: 6480)

In [6]:
from astropy import units as u
from astropy.coordinates import SkyCoord

ptf_ra = ptf_lcs['ra'].data.data
ptf_dec = ptf_lcs['dec'].data.data
print(len(np.unique(ptf_ra)))

# Match SDSS to PTF, since there is less PTF than SDSS 

# SDSS 9258 DR9 QSO 
File = '../../CRTS_PROJECT/catalogs_SDSS/DB_QSO_S82.dat'
colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']
SDSS_DB_QSO= Table.read(File, format='ascii', names = colnames)
sdss_catalog = SkyCoord(ra=SDSS_DB_QSO['ra']*u.degree, 
                        dec=SDSS_DB_QSO['dec']*u.degree)
ptf_catalog = SkyCoord(ptf_ra*u.degree,ptf_dec*u.degree, 
                       frame='icrs') 
idx, sep2d, dist3d = sdss_catalog.match_to_catalog_sky(ptf_catalog) 
SDSS_DB_QSO['d2d_arcsec']=sep2d.value*3600


# take the PTF table by matching indices 
ptf =  Table([ptf_ra[idx], ptf_dec[idx], ptf_lcs['oid'].data.data[idx]],
            names = ['ra_sdss', 'dec_sdss', 'ptf_oid'])

# combine 
sdss_ptf = hstack([SDSS_DB_QSO, ptf])

# only save those that are well matchewd ... 
matchRadiusArcsec = 0.5
fname = 'SDSS_DB9_PTF_QSO_database.txt'
mask = sdss_ptf['d2d_arcsec'] < matchRadiusArcsec
print('Of ', len(SDSS_DB_QSO) , ' SDSS DB9 QSO, ', 
      np.sum(mask), 
    ' has a PTF match within ', matchRadiusArcsec, ' arcseconds ')

print('We save these to ', fname)
sdss_ptf[mask].write(fname, format='ascii', overwrite=True)

6471
Of  9258  SDSS DB9 QSO,  6455  has a PTF match within  0.5  arcseconds 
We save these to  SDSS_DB9_PTF_QSO_database.txt


### SDSS-CRTS <a id='sdss_crts'></a>
[go to top](#toc)

NOTE : The SDSS-CRTS cross-matched catalog that was used before for the CRTS_PROJECT paper in  ../../CRTS_PROJECT/data_products/CRTS_SDSS_combined_QSO_catalog.dat ONLY  cross-matched those CRTS QSO that had more than 10 epochs per day. We don't need to impose that constraint here, so quickly do the cross-match again.... 

The code below takes ~2 mins to run - the results are saved . 

In [165]:
dir_in = '../../CRTS_PROJECT/raw_LC_CRTS/QSO_CRTS/'
QSO_arr = np.array(os.listdir(dir_in))
print('We have %d CRTS qso files to start with ' % len(QSO_arr))

# ignore the two files that are not lightcurves
a = np.array([len(entry) for entry in QSO_arr])
QSO_list = list(QSO_arr)
QSO_list.remove('message.txt')
QSO_list.remove('file.list')

c = 0 
empty_files = []
for obj in QSO_list:
    address = dir_in + obj
    f = open(address, 'r')
    if f.readline() == '' :
        c += 1
        empty_files.append(obj)
        
print('We have %d empty files' % c)    

crts_qso = np.array(QSO_list)[~np.in1d(QSO_list, empty_files)]  
print('We are left with ', len(crts_qso), 'non-empty files ... ')

# grab ra, dec from filenames 
ra_crts = np.array([coord[:9] for coord in crts_qso])
dec_crts = np.array([coord[9:-4] for coord in crts_qso])

# make into AstroPy - digestible format.. 
ra_crts_split = [ra[:2] + ' '+ ra[2:4] + ' '+ra[4:] for ra in ra_crts]
dec_crts_split = [dec[:3]+ ' '+ dec[3:5]+ ' ' + dec[5:] for dec in dec_crts]

# combining the two.. 
radec_crts_split = np.zeros_like(ra_crts_split, dtype='U32')
for i in range(len(ra_crts_split)):
    radec_crts_split[i] = ra_crts_split[i] + ' ' + dec_crts_split[i]
    
crts_catalog = SkyCoord(radec_crts_split, frame='icrs', unit=(u.hourangle, u.deg))

# Check how many  of 9258  SDSS S82 DR9 QSOs have CRTS matches 
File = '../../CRTS_PROJECT/catalogs_SDSS/DB_QSO_S82.dat'
colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']
SDSS_DB_QSO= Table.read(File, format='ascii', names = colnames)
sdss_catalog = SkyCoord(ra=SDSS_DB_QSO['ra']*u.degree, 
                        dec=SDSS_DB_QSO['dec']*u.degree)

# better : match SDSS to CRTS,  
# because there is less 
idx, sep2d, dist3d = sdss_catalog.match_to_catalog_sky(crts_catalog) 
SDSS_DB_QSO['d2d_arcsec']=sep2d.value*3600

crtsRA = crts_catalog[idx].ra.deg
crtsDEC = crts_catalog[idx].dec.deg
crtsID = crts_qso[idx]
crts = Table([crtsID, crtsRA, crtsDEC], names=['crtsID', 'crtsRA', 'crtsDEC'])

sdss_crts = hstack([SDSS_DB_QSO, crts])

matchRadiusArcsec = 0.5
fname = 'SDSS_DB9_CRTS_QSO_database.txt'
mask = sdss_crts['d2d_arcsec'] < matchRadiusArcsec
print('Of ', len(SDSS_DB_QSO) , ' SDSS DB9 QSO, ', 
      np.sum(mask), 
    ' has a CRTS match within ', matchRadiusArcsec, ' arcseconds ')

print('We save these to ', fname)
sdss_crts[mask].write(fname, format='ascii', overwrite=True)

We have 7933 CRTS qso files to start with 
We have 179 empty files
We are left with  7752 non-empty files ... 
Of  9258  SDSS DB9 QSO,  7737  has a CRTS match within  0.5  arcseconds 
We save these to  SDSS_DB9_CRTS_QSO_database.txt


##  SDSS,CRTS,PTF,PS1: inner join <a id='combine'></a>

Read in all catalogs ...

In [4]:
sdss_ps1 = Table.read('SDSS_DB9_PS1_QSO_database.txt', format='ascii')
sdss_ptf = Table.read('SDSS_DB9_PTF_QSO_database.txt', format='ascii')
sdss_crts = Table.read('SDSS_DB9_CRTS_QSO_database.txt', format='ascii')
print(len(sdss_ps1))
print(len(sdss_ptf))
print(len(sdss_crts))

9248
6455
7737


Join all tables (inner join)

In [5]:
sdss_ptf_crts = join(sdss_ptf, sdss_crts, keys='dbID')
sdss_ptf_crts_ps1 = join(sdss_ptf_crts, sdss_ps1, keys='dbID')
print(len(sdss_ptf_crts_ps1))

6444


Select only IDs from various surveys ... 


In [6]:
sdss_ptf_crts_ps1_ = sdss_ptf_crts_ps1[['dbID', 'ptf_oid','crtsID', 'ps1id']]

### Combine light curves 

In [53]:
# All ptf light cuerves are extracted from this big 
# IPAC table : read only once 
ptf_lcs =  Table.read('../../CRTS_PROJECT/IPAC_output_tables/\
Ipac_PTF_lightcurves_QSO_S82.tbl', 
                     format='ipac')

In [173]:
# Next step: loop  over these objects according to SDSS dbID

# SDSS colnames 
colnames = []
for f in 'ugriz':
    colnames.append(f+'mjd')
    colnames.append(f+ 'mag')
    colnames.append(f+'err')
colnames.append('RA')
colnames.append('DEC')

# skip those that are already saved ...
outDir = '../data_products/SDSS-PTF-CRTS-PS1_/'

In [175]:
def combine_lightcurves(row, sdss_ptf_crts_ps1_ = sdss_ptf_crts_ps1_):
    sdss_dir = '../../CRTS_PROJECT/raw_LC_SDSS/QSO_S82/'
    sdss_name = str(sdss_ptf_crts_ps1_[row]['dbID'].astype(int))
    
    print(row, sdss_name)
    
    sdss_lc = Table.read(sdss_dir+sdss_name, format='ascii',names=colnames)
    
    # recast the sdss_lc from umag,umjd,uerr,  gmag,gmjd,gerr   
    # into   mag,mjd,err,filter.... 

    # this object will hold all photometry from all surveys 
    lc  = Table()
    for f in 'ugriz':
        single = Table(sdss_lc[[f+'mag', f+'err', f+'mjd']], 
                       names = ['mag', 'magerr', 'mjd'])
        # don't store epochs  with no measurement in a given filter .... 
        single = single[single['mag'].data > 0] 
        single['filter'] = f
        single['survey'] = 'sdss'
        lc = vstack([lc,single])
    
    # add PTF info  
    ptf_name = sdss_ptf_crts_ps1_[row]['ptf_oid'].astype(int)
    ptf_lc = ptf_lcs[ptf_lcs['oid'].data.data == ptf_name]

    # PTF uses g, and R : g-band during dark time, and R-band during bright time
    # (1=g-band, 2=R-band). Majority is g-band (12 000 epochs), the rest is 
    # R-band (147 000 epochs). 
    
    ptf_lc_ = Table(ptf_lc['obsmjd', 'mag_autocorr', 'magerr_auto','fid'],
                    names=['mjd', 'mag', 'magerr','filter'])
    ptf_lc_['filter'] = ptf_lc_['filter'].astype(str)
    ptf_lc_['survey'] = 'ptf'

    # stack that
    lc = vstack([lc,ptf_lc_])

    # add CRTS info  : white light 
    dir_in = '../../CRTS_PROJECT/raw_LC_CRTS/QSO_CRTS/'
    crts_name = sdss_ptf_crts_ps1_[row]['crtsID']
    crts_lc = Table.read(dir_in+crts_name, format='ascii', 
                        names=['mjd', 'mag', 'magerr', '0'])
    crts_lc_ = crts_lc[['mjd', 'mag', 'magerr']]
    crts_lc_['filter'] = 'w'
    crts_lc_['survey'] = 'crts'

    # stack that
    lc = vstack([lc,crts_lc_])

    # add PS1  # extreme PIA because some rows simply 
    # don't have filter info! very annopying;...
    ps1_dir = '../raw_data/schneider/'
    ps1_name= sdss_ptf_crts_ps1_[row]['ps1id']
    print(ps1_name)
    fname = ps1_dir + ps1_name
    f = open(fname, 'r')
    lines = f.readlines()
    f.close()
    lines_strip = [l.strip() for l in lines]
    lines_split = [l.split() for l in lines_strip]

    mjd = [] ; mag = [] ; magerr = [] ; filtr = []

    for line in lines_split[1:]:
        if len(line) > 3 : 
            mjd.append(float(line[0]))
            mag.append(float(line[1]))
            magerr.append(float(line[2]))
            filtr.append(line[3])
    if len(mjd) > 0 :  # there may be light curves that have no good measurement
        ps1_lc_ = Table([np.asarray(mjd), np.asarray(mag),
                       np.asarray(magerr), np.asarray(filtr)],
                      names=['mjd', 'mag', 'magerr', 'filter'])
        ps1_lc_['survey'] = 'ps1'
        # stack that
        lc = vstack([lc,ps1_lc_])

    # Save the light curve , only the good obs ...
    lc.write(outDir+sdss_name+'.txt', format='ascii', overwrite='True')

done_lcs = os.listdir(outDir)
for row in range(len(sdss_ptf_crts_ps1_)):
    sdss_name = str(sdss_ptf_crts_ps1_[row]['dbID'].astype(int))
    if sdss_name+'.txt' not in done_lcs : 
        combine_lightcurves(row)
    

0 233
000119.50+001038.0
1 1018
000527.52-000556.2
2 1327
000107.52+004113.1
3 1540
001025.90+005447.6
4 1580
000640.67+003838.2
5 2113
001924.06-001635.0
6 2306
001606.24+002841.2
7 2401
000654.10-001533.4
8 2860
001542.39+001547.2
9 2866
001611.08-001512.4
10 2867
001527.82-003527.4
11 2953
001543.73-002648.1
12 3035
001602.63-002035.5
13 4370
001005.04+005141.4
14 4600
001526.52+001813.1
15 4671
001841.81-010832.5
16 5036
001854.25-010202.2
17 5429
001720.93-001225.4
18 5524
001824.12-010007.9
19 5626
001426.96-000306.7
20 5993
001116.43+002626.4
21 7838
000431.88+003155.1
22 8029
001807.02+002244.6
23 8442
001731.70+004910.1
24 9321
000512.11+001313.1
25 9836
001735.69-011325.0
26 9837
000531.41+001455.9
27 9872
001110.75+003029.1
28 10579
001016.49+001227.6
29 11021
001759.83-000548.2
30 11906
001412.93-002102.6
31 12291
001050.95-003133.2
32 12317
001919.31+010152.2
33 12417
001628.57+010722.9
34 13318
001252.46+002831.9
35 14323
000717.82-005137.7
36 14706
001602.40-001225.0
37 

000933.34+004414.7
287 122682
001135.06-011421.9
288 123137
001330.57+001814.2
289 123189
001443.69-011206.4
290 123709
001829.01+003515.7
291 123909
001626.54+003632.4
292 124689
000945.46+001337.0
293 125207
001534.67-011118.8
294 125388
000451.86-001203.7
295 125422
000717.73-002811.3
296 126986
001322.83+011348.2
297 127248
001920.12-002237.4
298 127330
001044.42+003344.7
299 127549
001535.71-001114.3
300 127950
001800.60-011523.0
301 128178
001645.49-004357.4
302 128761
000639.16+002820.9
303 128895
000912.69+010919.0
304 129274
001903.27-004223.4
305 129550
000737.13+002003.2
306 129563
000539.29-010243.8
307 129859
001957.60+003936.2
308 129886
001904.83+003436.5
309 130405
001438.28-010750.1
310 130457
001204.63-004113.7
311 130965
000801.72+004613.3
312 131196
001839.83-001303.0
313 131280
000637.89-001842.4
314 131569
001404.45+011237.4
315 131901
001545.36+000822.6
316 132784
000051.56+001202.5
317 133537
001416.25-011355.9
318 133895
001536.78+003757.3
319 134148
001619.22+

005152.20+010319.4
562 219249
004312.70+005605.0
563 219445
004448.36+010338.5
564 219531
005346.48+005343.6
565 219647
005126.02+004745.7
566 219953
004038.13+000312.8
567 220111
004336.39+010339.9
568 220391
004351.58+005501.8
569 220639
005614.81+010333.0
570 220661
005257.98+002742.6
571 220933
005824.75+004113.3
572 221006
005142.20+002129.0
573 221567
005232.43+005123.0
574 221975
005121.26+004521.6
575 222099
004634.06+003156.9
576 222220
004439.33+001822.8
577 222286
004806.05-010321.6
578 223324
004647.37-005100.1
579 223561
004759.86-005153.9
580 223661
005814.31+011530.2
581 223763
004323.43-001552.4
582 223800
005946.66+000255.0
583 224793
005154.67-001922.4
584 224886
004135.65-002632.7
585 225048
005722.30+001644.6
586 225080
005809.13+003118.2
587 225538
004715.88-003644.0
588 226040
005319.94+001826.0
589 226065
004420.11+001044.1
590 226283
004927.65+002542.5
591 227097
005132.59+005834.8
592 227300
004538.26+003016.4
593 227585
005308.64-011513.8
594 227805
004557.35-

233154.66-002217.6
837 313252
233612.39-004759.4
838 313478
232512.65+001025.9
839 313705
232530.48+005357.7
840 313854
233414.15+011507.7
841 314308
232102.44+001314.2
842 314342
232537.33-004246.5
843 314358
233155.01-011515.0
844 314443
233446.53+002721.0
845 314594
232538.01+005719.9
846 314829
232343.99-010037.4
847 315065
232300.69+003555.9
848 315981
232745.46-001038.9
849 316256
232505.23-005350.5
850 316394
232554.37-001823.8
851 317251
232824.54-003658.4
852 317339
232341.53+002723.9
853 317553
233056.43+003954.1
854 317716
233149.48+000719.5
855 319057
233239.25-010725.9
856 319570
232632.89-000326.6
857 319849
233121.81+003807.4
858 320427
232600.46-002029.3
859 320428
233246.03+004213.1
860 320440
233700.72+005456.7
861 320793
232555.33+004109.9
862 321158
232529.43-004735.0
863 321591
233717.24-002109.0
864 322070
233051.14+004936.1
865 322151
232742.67+000653.9
866 323193
232640.95-001347.1
867 323357
232557.46-005458.3
868 324188
232416.83+001404.3
869 325449
233018.84+

1107 408726
233132.28-004944.9
1108 408727
233610.47-005814.8
1109 409055
232515.93+002229.1
1110 409308
233611.57-003042.6
1111 409574
232034.78+000339.0
1112 409834
232056.98-005342.7
1113 410000
233525.65+003212.1
1114 410224
233549.61+003208.7
1115 410232
233317.38-002303.4
1116 411210
232907.12+003416.5
1117 411219
233517.82-004927.0
1118 411323
232434.45-011345.1
1119 411637
232836.52+011057.6
1120 411719
233713.51+000730.3
1121 411887
232712.98-003618.3
1122 411899
232342.02+005201.4
1123 412056
232707.28+000352.9
1124 412290
232951.44+000942.7
1125 412529
233006.65-010245.3
1126 413924
233750.68-001513.3
1127 415004
232428.36-005244.3
1128 416703
233512.44-002857.9
1129 417391
230228.95+002249.2
1130 417694
230015.18+010520.0
1131 418096
231605.72-011218.4
1132 418100
230524.46+005209.7
1133 418783
231341.06-010007.4
1134 419768
230946.14+000048.8
1135 419871
230045.18+002356.4
1136 420130
231759.63-000733.2
1137 420389
231047.98-003319.3
1138 420758
230347.27+011011.7
1139 420

...

031721.02-002637.2
5257 3134711
030206.77-000121.3
5258 3135035
030121.39+001726.3
5259 3136044
031435.72-003022.3
5260 3136183
031001.87-001002.0
5261 3137806
031134.48+001723.1
5262 3138327
031314.09-000913.7
5263 3138341
030505.90-000616.5
5264 3138839
031814.50-005647.2
5265 3139113
030322.79-004726.8
5266 3139151
030543.44-010622.1
5267 3139388
031249.02+005042.6
5268 3139560
030916.05+001834.4
5269 3139757
031501.20-004750.7
5270 3140579
030758.75+004259.0
5271 3140652
031731.98+001010.4
5272 3140841
031142.34+010355.1
5273 3141543
031325.57-003801.0
5274 3141804
031206.49+000449.6
5275 3142112
030047.57+000312.8
5276 3143448
030627.51-001816.3
5277 3144898
031237.56+004511.3
5278 3145609
031019.95+010111.5
5279 3146268
030242.82+000715.7
5280 3146376
030025.23+003224.2
5281 3147102
031846.13-005622.8
5282 3147340
031345.03-000720.2
5283 3147968
031318.66+003623.9
5284 3148142
031403.31-010408.2
5285 3148297
030220.31-002222.7
5286 3148867
031845.17-001845.3
5287 3148986
031620.5

204456.58+000658.7
5514 3706717
204538.96-005115.5
5515 3707189
204142.90+011029.0
5516 3710283
204957.77-004949.3
5517 3711085
205745.53+004652.0
5518 3718907
204402.83-000014.9
5519 3720533
032228.98-005628.6
5520 3727819
032214.41+005655.7
5521 3728258
032405.19-010140.5
5522 3729653
032400.51+004924.8
5523 3731788
032504.59-000652.1
5524 3734237
032117.64-004000.2
5525 3734258
032310.61+005417.2
5526 3735893
032335.80-005233.3
5527 3737743
032303.20+011258.8
5528 3737948
032547.04+001945.4
5529 3739176
032256.08-010808.9
5530 3741204
032118.82+002134.5
5531 3741594
032122.36-010600.5
5532 3742361
032218.69+004512.4
5533 3743328
032505.57-003445.5
5534 3745998
032427.38+003538.1
5535 3746591
032113.31+003909.8
5536 3749360
032450.56+010329.9
5537 3751346
032438.97+002326.9
5538 3751695
032246.82-005148.9
5539 3753388
032211.16-000035.3
5540 3753664
032309.00-005437.5
5541 3753915
033939.74-001226.8
5542 3755320
032009.03+003404.2
5543 3755470
032329.49+003128.3
5544 3756091
032324.3

234954.59-010954.8
5771 3918474
235008.88-002912.5
5772 3918643
234016.28+001111.3
5773 3918668
235926.26-004750.4
5774 3920272
235148.95+003218.0
5775 3920512
234541.56+010818.2
5776 3920780
235712.23-001102.8
5777 3921679
235605.48-000436.1
5778 3921928
234905.01+002855.0
5779 3921978
234921.56-005915.1
5780 3922126
235523.25+005150.7
5781 3922585
235656.16-011425.6
5782 3923134
235820.41+002659.7
5783 3923202
234928.83-010909.6
5784 3923317
234845.01+002237.8
5785 3923964
235352.50-001608.6
5786 3924315
235643.35+001428.0
5787 3924342
235358.14-003542.5
5788 3924460
234139.92+003644.2
5789 3924499
235106.03+001739.0
5790 3925045
234231.06+000618.2
5791 3925117
235219.08-000012.1
5792 3925196
234912.19+001656.6
5793 3925818
234619.37+000526.3
5794 3926085
234933.04+011524.1
5795 3926101
235313.22+000800.6
5796 3926449
235718.14-011529.0
5797 3926646
234142.04+004949.3
5798 3926678
235118.78+002556.7
5799 3926683
235735.86-000131.8
5800 3926779
235634.70-004503.8
5801 3926895
235948.0

6029 4012689
003945.76-010032.4
6030 4012869
003149.99-001433.1
6031 4013234
002146.71-004847.9
6032 4013494
002339.89+003348.9
6033 4014181
003640.22+010848.2
6034 4014674
003358.67+004855.4
6035 4014818
002847.68+000424.1
6036 4015671
003652.64+000826.7
6037 4015806
003503.77+001641.6
6038 4016163
002239.53-010741.2
6039 4016190
003913.87-005909.0
6040 4017321
002335.94+001227.9
6041 4018361
002334.20-002128.1
6042 4018385
002416.86-002049.6
6043 4018639
003420.24-004420.0
6044 4018877
003251.46-002748.0
6045 4019194
003542.51+002200.2
6046 4019717
002618.83+001100.9
6047 4019869
002331.21-011045.6
6048 4021063
003900.41+010725.9
6049 4021307
002238.06-001026.8
6050 4021530
003500.65-002505.2
6051 4022235
002023.18-001110.6
6052 4023056
002633.94-011119.4
6053 4023205
002440.99+004557.7
6054 4023246
002758.73-001923.8
6055 4024683
003138.17+003333.2
6056 4024726
003203.48+011524.7
6057 4024948
002322.71-004829.3
6058 4025002
002237.90+000519.0
6059 4025679
002333.77-003505.5
6060 402

6286 4112275
002056.80+001147.5
6287 4112395
003554.17-001353.0
6288 4112748
003551.88-000533.4
6289 4113127
002005.69-005016.3
6290 4113243
003533.63+005639.2
6291 4114672
003256.99-002611.3
6292 4115856
003703.11-001738.9
6293 4115974
002752.39+002615.6
6294 4116243
002422.04+004610.6
6295 4116418
003410.31+002349.9
6296 4117334
002332.34-011444.2
6297 4118049
003940.23+004241.5
6298 4118266
002926.15+002623.8
6299 4118324
003517.95+004333.7
6300 4118380
003758.81+000625.3
6301 4118411
002303.15+011533.6
6302 4118897
002910.87-003527.6
6303 4118904
003113.04+001020.3
6304 4119352
002019.68+005821.9
6305 4119455
003724.79+001103.5
6306 4121989
002642.18-011243.2
6307 4122526
003108.65-003905.8
6308 4122623
002609.07-003749.3
6309 4122790
002833.55+000715.4
6310 4123583
002059.32-010132.0
6311 4123955
002949.10+011248.6
6312 4124042
003628.11-002949.7
6313 4124306
003415.77-000030.8
6314 4124447
002945.28+003638.5
6315 4124707
003923.22-003117.0
6316 4124811
003153.84+005916.1
6317 412